In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, r2_score
import os, fnmatch
from sklearn.cluster import KMeans
from sklearn.preprocessing import PolynomialFeatures 
import seaborn as sns
%matplotlib inline
from keras.layers import Input, Dense
from keras.models import Model
# from keras.op
from keras import optimizers
from keras import regularizers
from PIL import Image
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import keras
from keras.models import load_model
from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import os
import pickle
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array

In [6]:
images_path = "C:/Users/Ghevond/Desktop/Specs3/"
a_list = []
data_train_names = os.listdir(images_path)
pattern = "*.tiff"
for entry in data_train_names:
    if fnmatch.fnmatch(entry, pattern):
            a_list.append(images_path + entry)

In [7]:
print(len(a_list))

610267


In [8]:
def split_list(the_list, chunk_size):
    result_list = []
    while the_list:
        result_list.append(the_list[:chunk_size])
        the_list = the_list[chunk_size:]
    return result_list

list_of_chunks = split_list(a_list, 50000)

In [9]:
print(len(list_of_chunks))

13


In [31]:
images_data = list_of_chunks[1]

In [32]:
len(images_data)

50000

In [33]:
max_width = 40
max_height = 140

In [34]:
for i in range(10):
    data = images_data[i]
    print(data)
    

C:/Users/Ghevond/Desktop/Specs3/fbs1243M10884_1371_4647_11_88_.tiff
C:/Users/Ghevond/Desktop/Specs3/fbs1243M10885_1368_4647_11_81_.tiff
C:/Users/Ghevond/Desktop/Specs3/fbs1243M10886_679_4647_14_126_.tiff
C:/Users/Ghevond/Desktop/Specs3/fbs1243M10887_403_4647_11_109_.tiff
C:/Users/Ghevond/Desktop/Specs3/fbs1243M10888_6952_4646_20_134_.tiff
C:/Users/Ghevond/Desktop/Specs3/fbs1243M10889_2981_4646_11_95_.tiff
C:/Users/Ghevond/Desktop/Specs3/fbs1243M1088_8279_9483_15_50_.tiff
C:/Users/Ghevond/Desktop/Specs3/fbs1243M10890_2577_4646_12_61_.tiff
C:/Users/Ghevond/Desktop/Specs3/fbs1243M10891_7750_4645_18_122_.tiff
C:/Users/Ghevond/Desktop/Specs3/fbs1243M10893_764_4645_11_133_.tiff


In [35]:
images_list = []
ind = 0
for i in range(len(images_data)):
    im = Image.open(images_data[i])
    arr = np.array(im)#.flatten()
    
    arr=(arr-arr.min())/(arr.max()-arr.min())
    #arr = arr / 0xffff
    images_list.append(arr)

In [ ]:
for i in range(len(images_list)):
    s = images_list[i].shape
    d_width = (max_width - s[1])
    d_height = (max_height - s[0])
    
    d_top = int(d_height / 2)
    d_bottom = int(d_height - d_top)
    
    d_left = int(d_width / 2)
    d_right = int(d_width - d_left)
    #print(d_top, d_bottom, d_left, d_right)
    
    arr = images_list[i]
    for l in range(d_left):
        arr = np.insert(arr, 0, 0, axis = 1)
    
    for r in range(d_right):
        b = np.zeros((s[0],1))
        arr = np.append(arr, b, axis = 1)
    
    for t in range(d_top):
        arr = np.insert(arr, 0, 0, axis = 0)
    
    for b in range(d_bottom):
        b = np.zeros((1, arr.shape[1],))
        arr = np.append(arr, b, axis = 0)
    
    images_list[i] = arr.flatten()

In [ ]:
width = max_width
height = max_height
print(width, height)
plt.imshow(images_list[6859].reshape(140, 40))

In [ ]:
from tensorflow.python.client import device_lib
import tensorflow as tf
import keras
#print(device_lib.list_local_devices())

config = tf.ConfigProto( device_count = {'GPU': 0 , 'CPU': 8} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

In [ ]:
images_np = np.array(images_list)

In [ ]:
print(images_np.shape)

In [ ]:
images_np = images_np.reshape(images_np.shape[0], 140, 40, 1)

In [ ]:
print(images_np.shape)

In [22]:
from keras.models import load_model
from keras.models import model_from_json
import json

with open('model_best.json','r') as f:
    model_json = json.load(f)

s1 = json.dumps(model_json)
    
model = model_from_json(s1)
model.load_weights('model_best.h5')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.






In [23]:
model

In [ ]:
arr_res = model.predict(images_np)

In [ ]:
np.max(arr_res[10])

In [ ]:
arr_class = []
for i in range(len(arr_res)):
    max_p = np.max(arr_res[i])
    res_class = -1;
    if(max_p > 0.75):
        res_class = np.argmax(arr_res[i])
    
    arr_class.append(res_class)

In [ ]:
print(arr_res[10], arr_class[10])

In [ ]:
out_path = "C:/Users/Ghevond/Desktop/Astro/TEST/Specs3/"
for i in range(len(arr_class)):
    current_image = images_data[i]
    class_string = str(arr_class[i])
    if(class_string == "-1"): 
        continue
    new_path = out_path + class_string + "/" + current_image.split("/")[5]
    #print(current_image, class_string, new_path)
    #break
    im = Image.open(current_image)
    im.save(new_path)
    